In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import os
import cv2

import sys
sys.path.append('..')

from shared.data import KITTIData,  VisualOdometry, draw_matches, draw_keypoints
from shared.tools import find_max_clique

%matplotlib widget

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
# https://github.com/matplotlib/ipympl

In [ ]:
DATASET_DIR = os.path.join('../', 'data/KITTI/dataset')
dataset = KITTIData(DATASET_DIR)
gt_poses = dataset.get_poses()

In [ ]:
poses = [
    np.eye(4)
]

from concurrent.futures import ProcessPoolExecutor

def process_transform(idx):
    print(f'Start idx: {idx}')
    c_l_img, c_r_img = dataset.get_color_images(frame_idx)
    n_l_img, n_r_img = dataset.get_color_images(frame_idx+1)
    Q_left = dataset.get_color_left_Q_matrix()
    gt_transform = dataset._get_transform_mtrx(frame_idx)
    
    vo = VisualOdometry()
    c_depth_frame = vo.process_depth(c_l_img, c_r_img, Q_left)
    n_depth_frame = vo.process_depth(n_l_img, n_r_img, Q_left)

    c_feats, n_feats = vo.get_features(c_l_img, n_l_img)

    ### Get 3D points 
    c_pnts_3d, c_ft_idxs = vo.reproject_2d_to_3d_points(c_feats, c_depth_frame)
    n_pnts_3d, n_ft_idxs = vo.reproject_2d_to_3d_points(n_feats, n_depth_frame)

    ft_idxs = c_ft_idxs & n_ft_idxs

    c_pnts_3d = c_pnts_3d[ft_idxs]
    n_pnts_3d = n_pnts_3d[ft_idxs]

    ### Filter 
    cl_idxs, _ = vo.max_clique_filter(c_pnts_3d, n_pnts_3d)
    c_pnts_3d = c_pnts_3d[cl_idxs]
    n_pnts_3d = n_pnts_3d[cl_idxs]

    C_left, _ = dataset.get_color_С_matrix()
    P_left = np.hstack((C_left, np.transpose([[0, 0, 0]])))

    c_pnts_2d = vo.reproject_3d_to_2d(c_pnts_3d, P_left)
    n_pnts_2d = vo.reproject_3d_to_2d(n_pnts_3d, P_left)

    transform = vo.get_transform(c_pnts_3d, n_pnts_3d, C_left)
    print(f'Transform for idx {idx} done')
    
    return transform
    
with ProcessPoolExecutor(8) as ex:
    transforms = ex.map(process_transform, range(len(gt_poses)))

for transform in transforms:
    n_pose = vo.get_next_pose(transform, poses[-1])
    poses.append(n_pose)


In [ ]:
from shared import common

common.plot_trajectories(gt_poses, poses)